In [202]:
from sympy import *
from scipy.sparse import coo_array
import numpy as np

output_eqs = []

In [203]:
from sympy.solvers.solveset import linsolve

def elim(eqs, z):
    """return eqs with parameter z eliminated from each equation; the first
    element in the returned list will be the definition of z that was used
    to eliminate z from the other equations.

    Examples
    ========

    >>> eqs = [Eq(2*x + 3*y + 4*z, 1),
    ...        Eq(9*x + 8*y + 7*z, 2)]
    >>> eliminate(eqs, z)
    [Eq(z, -x/2 - 3*y/4 + 1/4), Eq(11*x/2 + 11*y/4 + 7/4, 2)]
    >>> Eq(y,solve(_[1], y)[0])
    Eq(y, -2*x + 1/11)
    """
    Z = Dummy()
    rv = []
    for i, e in enumerate(eqs):
        if z not in e.free_symbols:
            continue
        e = e.subs(z, Z)
        if z in e.free_symbols:
            break
        try:
            s = linsolve([e], Z)
            if s:
                zi = list(s)[0][0]
                rv.append(Eq(z, zi))
                rv.extend([eqs[j].subs(z, zi)
                    for j in range(len(eqs)) if j != i])
                return rv
        except ValueError:
            continue
    raise ValueError

def eliminate(eqset, vars) :
    """
    Calls the elim function on each variable in vars recursively.
    Thus eliminates all variables in vars from the list of equations eqset.
    """
    if len(vars)==0 :
        global output_eqs
        output_eqs = eqset
        print('Final solution -')
        for eq in eqset :
            print(eq)
        return
    var = vars.pop(0)
    try :
        tempsols = elim(eqset, var)
    except ValueError :
        print(f'Zero coeff variable - {var}')
        try : var = vars.pop(0)
        except IndexError :
            print('All variables eliminated')
        return eliminate(eqset, vars)
    else :
        print(f'Eliminated by equation - {tempsols[0]}')
        return eliminate(tempsols[1:], vars)

In [204]:
ABC = np.zeros((8, 8))

ABC_coeffs = []

for i in range(1, 256) :
    coeff = symbols(f'ABC{i:03d}')
    ABC_coeffs.append(coeff)
    arr = np.array([int(c) for c in f'{i:08b}'])
    mat = np.outer(arr, arr)

    ABC = ABC + coeff * mat

ABC = Matrix(ABC)
# ABC

In [205]:
BC = np.zeros((4, 4))

BC_coeffs = []

for i in range(1, 16) :
    coeff = symbols(f'oBC{i:03d}')
    BC_coeffs.append(coeff)
    arr = np.array([int(c) for c in f'{i:04b}'])
    mat = np.outer(arr, arr)

    BC = BC + coeff * mat

BC = Matrix(BC)
# BC

In [206]:
AB = np.zeros((4, 4))

AB_coeffs = []

for i in range(1, 16) :
    coeff = symbols(f'oAB{i:03d}')
    AB_coeffs.append(coeff)
    arr = np.array([int(c) for c in f'{i:04b}'])
    mat = np.outer(arr, arr)

    AB = AB + coeff * mat

AB = Matrix(AB)
# AB

In [207]:
CA = np.zeros((4, 4))

CA_coeffs = []

for i in range(1, 16) :
    coeff = symbols(f'oCA{i:03d}')
    CA_coeffs.append(coeff)
    arr = np.array([int(c) for c in f'{i:04b}'])
    mat = np.outer(arr, arr)

    CA = CA + coeff * mat

CA = Matrix(CA)
# CA

In [208]:
cyclic_swap = Matrix(
   [[1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1]] )

# cyclic_swap

In [209]:
BC_refs = ABC[0:4, 0:4] + ABC[4:8, 4:8]
CAB = cyclic_swap * ABC * cyclic_swap.T
AB_refs = CAB[0:4, 0:4] + CAB[4:8, 4:8]
BCA = cyclic_swap * CAB * cyclic_swap.T
CA_refs = BCA[0:4, 0:4] + BCA[4:8, 4:8]

In [210]:
index_arr = [(0, 0), (0, 1), (0, 2), (0, 3),
                     (1, 1), (1, 2), (1, 3),
                             (2, 2), (2, 3),
                                     (3, 3)]

eq_arr = []

for i in index_arr :
    eq_arr.append(BC[i[0], i[1]] - BC_refs[i[0], i[1]])
    eq_arr.append(AB[i[0], i[1]] - AB_refs[i[0], i[1]])
    eq_arr.append(CA[i[0], i[1]] - CA_refs[i[0], i[1]])

In [211]:
EABC = diag(ABC, diag(0, 0, 0, 0, 0, 0, 0, 0))
# EABC

In [212]:
row  = np.array(range(16))
col  = np.array([[i, i+8] for i in range(8)]).reshape(16)
data = np.ones(16, dtype=int)

cyclic_swap_4 = Matrix(coo_array((data, (row, col)), shape=(16, 16)).toarray())
# cyclic_swap_4

In [213]:
ABCE = cyclic_swap_4 * EABC * cyclic_swap_4.T
# ABCE

In [214]:
Toff = diag(1, 1, 1, 1, 1, 1, Matrix([[0, 1], [1, 0]]))
Toff_16 = diag(Toff, Toff)
# Toff_16

In [215]:
toffedABCE = Toff_16 * ABCE * Toff_16.T
# toffedABCE

In [216]:
toffedBCEA = cyclic_swap_4 * toffedABCE * cyclic_swap_4.T
# toffedBCEA

In [217]:
toffedEA = toffedBCEA[0:4, 0:4] + toffedBCEA[4:8, 4:8] + toffedBCEA[8:12, 8:12] + toffedBCEA[12:16, 12:16]
# toffedEA

In [218]:
EA = np.zeros((4, 4))

EA_coeffs = []

for i in range(1, 16) :
    coeff = symbols(f'oEA{i:03d}')
    EA_coeffs.append(coeff)
    arr = np.array([int(c) for c in f'{i:04b}'])
    mat = np.outer(arr, arr)

    EA = EA + coeff * mat

EA = Matrix(EA)
# EA

In [219]:
for i in index_arr :
    eq_arr.append(EA[i[0], i[1]] - toffedEA[i[0], i[1]])

# eq_arr.append(np.trace(EA) - np.trace(ABC))
    
len(eq_arr)
# eq_arr

40

In [220]:
eliminate(eq_arr, ABC_coeffs)

Eliminated by equation - Eq(ABC001, -ABC003 - ABC005 - ABC007 - ABC009 - ABC011 - ABC013 - ABC015 - ABC016 - 2*ABC017 - ABC018 - 2*ABC019 - ABC020 - 2*ABC021 - ABC022 - 2*ABC023 - ABC024 - 2*ABC025 - ABC026 - 2*ABC027 - ABC028 - 2*ABC029 - ABC030 - 2*ABC031 - ABC033 - ABC035 - ABC037 - ABC039 - ABC041 - ABC043 - ABC045 - ABC047 - ABC048 - 2*ABC049 - ABC050 - 2*ABC051 - ABC052 - 2*ABC053 - ABC054 - 2*ABC055 - ABC056 - 2*ABC057 - ABC058 - 2*ABC059 - ABC060 - 2*ABC061 - ABC062 - 2*ABC063 - ABC065 - ABC067 - ABC069 - ABC071 - ABC073 - ABC075 - ABC077 - ABC079 - ABC080 - 2*ABC081 - ABC082 - 2*ABC083 - ABC084 - 2*ABC085 - ABC086 - 2*ABC087 - ABC088 - 2*ABC089 - ABC090 - 2*ABC091 - ABC092 - 2*ABC093 - ABC094 - 2*ABC095 - ABC097 - ABC099 - ABC101 - ABC103 - ABC105 - ABC107 - ABC109 - ABC111 - ABC112 - 2*ABC113 - ABC114 - 2*ABC115 - ABC116 - 2*ABC117 - ABC118 - 2*ABC119 - ABC120 - 2*ABC121 - ABC122 - 2*ABC123 - ABC124 - 2*ABC125 - ABC126 - 2*ABC127 - ABC129 - ABC131 - ABC133 - ABC135 - ABC137 -

In [221]:
output_eqs

[-oAB004 - oAB005 - oAB006 - oAB007 - oAB008 - oAB009 - oAB010 - oAB011 - 2*oAB012 - 2*oAB013 - 2*oAB014 - 2*oAB015 + oBC002 + oBC003 + oBC006 + oBC007 + oBC008 + oBC009 + 2*oBC010 + 2*oBC011 + oBC012 + oBC013 + 2*oBC014 + 2*oBC015,
 oAB002 + oAB003 + oAB006 + oAB007 + oAB008 + oAB009 + 2*oAB010 + 2*oAB011 + oAB012 + oAB013 + 2*oAB014 + 2*oAB015 - oCA004 - oCA005 - oCA006 - oCA007 - oCA008 - oCA009 - oCA010 - oCA011 - 2*oCA012 - 2*oCA013 - 2*oCA014 - 2*oCA015,
 -oBC004 - oBC005 - oBC006 - oBC007 - oBC008 - oBC009 - oBC010 - oBC011 - 2*oBC012 - 2*oBC013 - 2*oBC014 - 2*oBC015 + oCA002 + oCA003 + oCA006 + oCA007 + oCA008 + oCA009 + 2*oCA010 + 2*oCA011 + oCA012 + oCA013 + 2*oCA014 + 2*oCA015,
 -oAB005 - oAB007 - oAB010 - oAB011 - oAB013 - oAB014 - 2*oAB015 + oBC003 + oBC007 + oBC011 + oBC012 + oBC013 + oBC014 + 2*oBC015,
 oAB003 + oAB007 + oAB011 + oAB012 + oAB013 + oAB014 + 2*oAB015 - oCA005 - oCA007 - oCA010 - oCA011 - oCA013 - oCA014 - 2*oCA015,
 -oBC005 - oBC007 - oBC010 - oBC011 - oBC

In [222]:
op_eq = []
for eq in output_eqs :
    substituted_eq = eq
    for var in EA_coeffs :
        substituted_eq = substituted_eq.subs(var, 0)
    if not eq == substituted_eq :
        op_eq.append(eq)

print(len(op_eq))
op_eq

8


[oEA010 + oEA011 + oEA014 + oEA015,
 oEA009 + oEA011 + oEA013 + oEA015,
 -oAB004 - oAB005 - oAB006 - oAB007 - oAB008 - oAB009 - oAB010 - oAB011 - 2*oAB012 - 2*oAB013 - 2*oAB014 - 2*oAB015 - oBC004 - oBC005 - oBC006 - oBC007 - oBC012 - oBC013 - oBC014 - oBC015 + oEA004 + oEA005 + oEA006 + oEA007 + oEA008 + oEA009 + oEA010 + oEA011 + 2*oEA012 + 2*oEA013 + 2*oEA014 + 2*oEA015,
 oEA006 + oEA007 + oEA014 + oEA015,
 oEA005 + oEA007 + oEA013 + oEA015,
 -oCA004 - oCA005 - oCA006 - oCA007 - oCA008 - oCA009 - oCA010 - oCA011 - 2*oCA012 - 2*oCA013 - 2*oCA014 - 2*oCA015 + oEA002 + oEA003 + oEA006 + oEA007 + oEA008 + oEA009 + 2*oEA010 + 2*oEA011 + oEA012 + oEA013 + 2*oEA014 + 2*oEA015,
 -oCA005 - oCA007 - oCA010 - oCA011 - oCA013 - oCA014 - 2*oCA015 + oEA003 + oEA007 + oEA011 + oEA012 + oEA013 + oEA014 + 2*oEA015,
 -oBC001 - oBC003 - oBC005 - oBC007 - oBC009 - oBC011 - oBC013 - oBC015 + oCA004 + oCA005 + oCA006 + oCA007 + oCA008 + oCA009 + oCA010 + oCA011 + 2*oCA012 + 2*oCA013 + 2*oCA014 + 2*oCA015

In [223]:
final_eq_set = []
varnum = [10, 11, 14, 15, 9, 13, 6, 7, 5]  # 1, 2, 3, 4, 8, 12 nonzero
for eq in op_eq :
    substituted_eq = eq
    for idx in varnum :
        var = EA_coeffs[idx-1]
        substituted_eq = substituted_eq.subs(var, 0)
    if not substituted_eq == 0 :
        final_eq_set.append(substituted_eq)

print(len(final_eq_set))
final_eq_set

4


[-oAB004 - oAB005 - oAB006 - oAB007 - oAB008 - oAB009 - oAB010 - oAB011 - 2*oAB012 - 2*oAB013 - 2*oAB014 - 2*oAB015 - oBC004 - oBC005 - oBC006 - oBC007 - oBC012 - oBC013 - oBC014 - oBC015 + oEA004 + oEA008 + 2*oEA012,
 -oCA004 - oCA005 - oCA006 - oCA007 - oCA008 - oCA009 - oCA010 - oCA011 - 2*oCA012 - 2*oCA013 - 2*oCA014 - 2*oCA015 + oEA002 + oEA003 + oEA008 + oEA012,
 -oCA005 - oCA007 - oCA010 - oCA011 - oCA013 - oCA014 - 2*oCA015 + oEA003 + oEA012,
 -oBC001 - oBC003 - oBC005 - oBC007 - oBC009 - oBC011 - oBC013 - oBC015 + oCA004 + oCA005 + oCA006 + oCA007 + oCA008 + oCA009 + oCA010 + oCA011 + 2*oCA012 + 2*oCA013 + 2*oCA014 + 2*oCA015 + oEA001 + oEA003 - oEA008 - oEA012]

In [229]:
for eq in list(linsolve(final_eq_set, EA_coeffs))[0] : print(eq)

oBC001 + oBC003 + oBC005 + oBC007 + oBC009 + oBC011 + oBC013 + oBC015 - oCA004 - 2*oCA005 - oCA006 - 2*oCA007 - oCA008 - oCA009 - 2*oCA010 - 2*oCA011 - 2*oCA012 - 3*oCA013 - 3*oCA014 - 4*oCA015 + oEA008 + 2*oEA012
oCA004 + oCA006 + oCA008 + oCA009 + 2*oCA012 + oCA013 + oCA014 - oEA008
oCA005 + oCA007 + oCA010 + oCA011 + oCA013 + oCA014 + 2*oCA015 - oEA012
oAB004 + oAB005 + oAB006 + oAB007 + oAB008 + oAB009 + oAB010 + oAB011 + 2*oAB012 + 2*oAB013 + 2*oAB014 + 2*oAB015 + oBC004 + oBC005 + oBC006 + oBC007 + oBC012 + oBC013 + oBC014 + oBC015 - oEA008 - 2*oEA012
oEA005
oEA006
oEA007
oEA008
oEA009
oEA010
oEA011
oEA012
oEA013
oEA014
oEA015


In [230]:
EA

Matrix([
[oEA008 + oEA009 + oEA010 + oEA011 + oEA012 + oEA013 + oEA014 + oEA015,                                     oEA012 + oEA013 + oEA014 + oEA015,                                     oEA010 + oEA011 + oEA014 + oEA015,                                     oEA009 + oEA011 + oEA013 + oEA015],
[                                    oEA012 + oEA013 + oEA014 + oEA015, oEA004 + oEA005 + oEA006 + oEA007 + oEA012 + oEA013 + oEA014 + oEA015,                                     oEA006 + oEA007 + oEA014 + oEA015,                                     oEA005 + oEA007 + oEA013 + oEA015],
[                                    oEA010 + oEA011 + oEA014 + oEA015,                                     oEA006 + oEA007 + oEA014 + oEA015, oEA002 + oEA003 + oEA006 + oEA007 + oEA010 + oEA011 + oEA014 + oEA015,                                     oEA003 + oEA007 + oEA011 + oEA015],
[                                    oEA009 + oEA011 + oEA013 + oEA015,                                     oEA005 + oEA007 + oEA013 

In [ ]:
# next step is to approach from the other way, i.e. eigenvector decomposition of density matrices and correlating them